In [ ]:
%pip install ortools

In [1]:
from ortools.sat.python import cp_model

def optimize_budget_with_weighted_quadratic_loss(current_allocations, income, scale=1000):
    """
    Optimize budget allocation while penalizing deviations from current allocations using a weighted quadratic loss.
    
    The loss for each category is:
      weight[cat] * (new_allocation[cat] - current_allocations[cat])^2
    where weight[cat] is computed as scale / (current_allocations[cat] + 1).
    
    Arguments:
      current_allocations: dict mapping category names (e.g., 'savings', 'housing', etc.) to their current values.
      income: total monthly take-home income.
      scale: a constant used to determine the weight for each category.
      
    Returns:
      A tuple (result, total_weighted_quad_loss) where:
        - result is a dict with the optimized allocation for each category,
        - total_weighted_quad_loss is the sum of the weighted quadratic losses.
    """
    model = cp_model.CpModel()
    
    # Define budget categories.
    categories = ['transport', 'food', 'housing', 'insurance', 'other_needs',
                  'investment', 'savings', 'total_needs', 'total_wants']
    
    # Compute a weight for each category based on its current allocation.
    # A lower current allocation gives a higher weight.
    weights = {}

    current_total_needs = 0
    for needs in ['transport', 'food', 'housing', 'insurance', 'other_needs']:
        current_total_needs += current_allocations.get(needs, 0)
    current_total_wants = income - current_total_needs - current_allocations.get('savings', 0)
    current_allocations['total_needs'] = current_total_needs
    current_allocations['total_wants'] = current_total_wants

    for cat in categories:
        current_value = current_allocations.get(cat, 0)
        weights[cat] = scale / (current_value + 1)
        # Round to an integer (ensuring a minimum weight of 1).
        weights[cat] = max(1, int(round(weights[cat])))
    
    variables = {}
    quad_vars = {}  # quadratic loss variable for each category.
    weighted_quad_terms = []
    
    # Create decision variables and compute weighted quadratic loss for each category.
    for cat in categories:
        # Decision variable for each category: allocation from 0 to income.
        variables[cat] = model.NewIntVar(0, income, cat)
        
        # Deviation variable: difference between the new allocation and the current allocation.
        dev = model.NewIntVar(-income, income, f"dev_{cat}")
        current_val = current_allocations.get(cat, 0)
        model.Add(dev == variables[cat] - current_val)
        
        # Quadratic loss: compute the square of the deviation.
        quad = model.NewIntVar(0, income**2, f"quad_{cat}")
        model.AddMultiplicationEquality(quad, [dev, dev])
        quad_vars[cat] = quad
        
        # The weighted quadratic term is weight[cat] * quad.
        weighted_term = weights[cat] * quad
        weighted_quad_terms.append(weighted_term)
    
    # --- Budget Constraints ---
    # For example, enforce that the sum of transport, food, housing, and insurance
    # is at most the allocated total_needs.
    model.Add(sum(variables[cat] for cat in ['transport', 'food', 'housing', 'insurance'])
              <= variables['total_needs'])
    
    # Total allocation constraint: total_needs, total_wants, and savings must sum to income.
    model.Add(sum(variables[cat] for cat in ['total_needs', 'total_wants', 'savings']) == income)
    
    # Hard constraints (adjust these as needed):
    model.Add(variables['savings'] >= int(0.2 * income))
    model.Add(variables['total_needs'] <= int(0.5 * income))
    model.Add(variables['total_wants'] <= int(0.3 * income))
    
    # --- Objective: minimize the total weighted quadratic loss ---
    model.Minimize(sum(weighted_quad_terms))
    
    # Solve the model.
    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    
    if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        result = {cat: solver.Value(variables[cat]) for cat in categories}
        total_weighted_quad_loss = sum(solver.Value(quad_vars[cat]) * weights[cat] for cat in categories)
        return result, total_weighted_quad_loss
    else:
        return None, None

# Example usage:
if __name__ == "__main__":
    current_allocations = {
        'transport': 200,
        'food': 1000,
        'housing': 1200,
        'insurance': 50,
        'other_needs': 50,
        'investment': 300,
        'savings': 1000,
    }
    income = 4000
    solution, loss = optimize_budget_with_weighted_quadratic_loss(current_allocations, income, scale=1000)
    if solution:
        print("Optimized Allocation:")
        for k, v in solution.items():
            print(f"  {k}: {v}")
        print("Total Weighted Quadratic Loss:", loss)
    else:
        print("No feasible solution found.")


load c:\repos\cs3263-project\.venv\Lib\site-packages\ortools\.libs\zlib1.dll...
load c:\repos\cs3263-project\.venv\Lib\site-packages\ortools\.libs\abseil_dll.dll...
load c:\repos\cs3263-project\.venv\Lib\site-packages\ortools\.libs\utf8_validity.dll...
load c:\repos\cs3263-project\.venv\Lib\site-packages\ortools\.libs\re2.dll...
load c:\repos\cs3263-project\.venv\Lib\site-packages\ortools\.libs\libprotobuf.dll...
load c:\repos\cs3263-project\.venv\Lib\site-packages\ortools\.libs\highs.dll...
load c:\repos\cs3263-project\.venv\Lib\site-packages\ortools\.libs\ortools.dll...
Optimized Allocation:
  transport: 160
  food: 800
  housing: 1000
  insurance: 40
  other_needs: 50
  investment: 300
  savings: 1333
  total_needs: 2000
  total_wants: 667
Total Weighted Quadratic Loss: 506667
